 EMOTION DETECTION FROM TEXT

In [ ]:
!pip3 install ktrain -q

     |████████████████████████████████| 25.3 MB 1.7 MB/s 
     |████████████████████████████████| 981 kB 59.9 MB/s 
     |████████████████████████████████| 2.8 MB 46.9 MB/s 
     |████████████████████████████████| 263 kB 64.0 MB/s 
     |████████████████████████████████| 1.2 MB 64.4 MB/s 
     |████████████████████████████████| 468 kB 69.3 MB/s 
     |████████████████████████████████| 22.3 MB 1.1 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 3.3 MB 56.2 MB/s 
     |████████████████████████████████| 880 kB 72.7 MB/s 
     |████████████████████████████████| 101 kB 14.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

Importing data

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/lukasgarbas/nlp-text-emotion/master/data/data_train.csv'
res = requests.get(url, allow_redirects=True)
with open('data_train.csv','wb') as file:
    file.write(res.content)
data_train = pd.read_csv('data_train.csv')

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/lukasgarbas/nlp-text-emotion/master/data/data_test.csv'
res = requests.get(url, allow_redirects=True)
with open('data_test.csv','wb') as file:
    file.write(res.content)
data_test = pd.read_csv('data_test.csv')

In [ ]:
X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

size of training set: 7934
size of validation set: 3393
joy        2326
sadness    2317
anger      2259
neutral    2254
fear       2171
Name: Emotion, dtype: int64


,Emotion,Text
0,neutral,There are tons of other paintings that I thin...
1,sadness,"Yet the dog had grown old and less capable , a..."
2,fear,When I get into the tube or the train without ...
3,fear,This last may be a source of considerable disq...
4,anger,She disliked the intimacy he showed towards so...
5,sadness,When my family heard that my Mother's cousin w...
6,joy,Finding out I am chosen to collect norms for C...
7,anger,A spokesperson said : ` Glen is furious that t...
8,neutral,Yes .
9,sadness,"When I see people with burns I feel sad, actua..."


In [ ]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}

# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

Data preprocessing

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


Training and validation

In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [ ]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
1323/1323 [==============================] - 827s 610ms/step - loss: 0.8914 - accuracy: 0.6581 - val_loss: 0.5603 - val_accuracy: 0.8014
Epoch 2/3
1323/1323 [==============================] - 806s 609ms/step - loss: 0.4321 - accuracy: 0.8515 - val_loss: 0.5169 - val_accuracy: 0.8161
Epoch 3/3
1323/1323 [==============================] - 806s 609ms/step - loss: 0.1734 - accuracy: 0.9456 - val_loss: 0.5857 - val_accuracy: 0.8226


In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

              precision    recall  f1-score   support

         joy       0.85      0.85      0.85       707
     sadness       0.78      0.83      0.80       676
        fear       0.87      0.83      0.85       679
       anger       0.80      0.80      0.80       693
     neutral       0.82      0.82      0.82       638

    accuracy                           0.82      3393
   macro avg       0.82      0.82      0.82      3393
weighted avg       0.82      0.82      0.82      3393



array([[599,  23,  16,  15,  54],
       [ 16, 559,  25,  55,  21],
       [ 20,  39, 562,  44,  14],
       [ 19,  66,  28, 551,  29],
       [ 47,  29,  14,  28, 520]])

Obtained accuracy:0.82

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['joy', 'sadness', 'fear', 'anger', 'neutral']

In [ ]:
import time 

message = 'I just broke up with my boyfriend'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: sadness (0.11)


In [ ]:
 predictor.save("/content/models/bert_model")